In [13]:
import struct
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import csv
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

In [117]:
# create a dataframe from csv file
def create_frame(file):
    df = pd.read_csv(file)
    return df

# create readable structure from txt file
def create_frame_info(fi_name):
    with open(fi_name) as f:
        frame_info = f.readlines()
    return frame_info

# create a list of ids present in a readable structure
def get_ids_from_txt(frame_info):
    ids = []
    for line in frame_info:
        a, rest = line.split(",", 1)
        ids.append(int(a))
    return(ids)

def process_batch(csvname, finame, npyname, vname):
    a = clean_csv(csvname, finame)
    truths = np.load(npyname)
    idx = get_frames_with_fish(vname, truths, a)
    b = remove_rows_by_index(a, idx)
    print("Kept {:d} lines based on npy T/F.".format(b.shape[0]))
    videoID = a.iloc[0][' video_id'][1:]
    return b

def clean_csv(csvfile, fifile):
    dframe = create_frame(csvfile)
    print("Created dataframe with {:d} rows.".format(dframe.shape[0]))
    frame_info = create_frame_info(fifile)
    print("Created frameinfo file with {:d} rows.".format(len(frame_info)))
    csvids = get_ids_from_frame(dframe)
    fiids = get_ids_from_txt(frame_info)
    diff = get_difference(csvids, fiids)
    final_frame = remove_rows(dframe, diff)
    print("Removed {:d} rows from dataframe. Final row count is {:d}.".format(len(diff), final_frame.shape[0]))
    return final_frame

def get_frames_with_fish(vname, npy, dframe):
    videoID = dframe.iloc[0][' video_id'][1:]
    vidcap = cv2.VideoCapture(vname)
    success,image = vidcap.read()
    count = 0
    rows_to_remove = []
    frame = 0
    while success:
        if (npy[count]):
            fid = str(list(full_df.index.values)[frame])
            imgname = "output/"+fid+".jpg"
            print(imgname)
            cv2.imwrite(imgname, image)     # save frame as JPG file
#             plt.imshow(image)
#             plt.show()
            frame+= 1
        else:
            rows_to_remove.append(count)
        success,image = vidcap.read()
        count += 1
    return rows_to_remove

# get all points from a contour and store as a list of tuples
def get_contour_points(contour):
    cont = contour[2:-1]
    tuples = cont.split(";")
    l = []
    for item in tuples:
        a, b = item.split(" ")
        l.append((int(a), int(b)))
    return l

# gets all contours form a dataframe. We can conveniently traverse this using detection id
def get_contours(dataframe):
    return dataframe[' contour']

# get IDs from dataframe
def get_ids_from_frame(frame):
    return frame['detection_id'].tolist()

#get set difference
def get_difference(a, b):
    return list(set(a).difference(set(b)))

def remove_rows(dframe, idlist):
    dframe = dframe.set_index("detection_id")
    dframe = dframe.drop(idlist, axis=0)
    return dframe

def remove_rows_by_index(dframe, idxlist):
    return dframe.drop(dframe.index[idxlist])

In [118]:
full_df = process_batch(full_path_and_name, frame_info_path, npy_path, video_path)

Created dataframe with 21 rows.
Created frameinfo file with 16 rows.
Removed 5 rows from dataframe. Final row count is 16.
output/432533639.jpg
output/432533642.jpg
output/432533647.jpg
Kept 3 lines based on npy T/F.


In [15]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [17]:
path_to_csvs = "/media/sam/Sam\'s Drive/SUBSET/SQL/ALLYEARS/"

In [ ]:
full_path_and_name = path_to_csvs + mynames[0][:-1]
print(full_path_and_name)
df = pd.read_csv(full_path_and_name)
df

In [25]:
videoID = df.iloc[0][4][1:]
video_name_fix = "summary_" + videoID + ".avi"
npy_name_fix = videoID + ".RESULT.npy"
frame_info_fix = "frame_info_" + videoID + ".txt"
videos_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
frame_info_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/f4k_extracted_image/output/summaries/"
npy_dir = "/media/sam/My Passport/FISH4KNOWLEDGE/final/"

In [26]:
video_path = find(video_name_fix, videos_dir)
frame_info_path = find(frame_info_fix, frame_info_dir)
npy_path = find(npy_name_fix, npy_dir)

In [79]:
full_df

,detection_component_id,recognition_component_id,fish_id,video_id,frame_id,bounding_box_x,bounding_box_y,bounding_box_w,bounding_box_h,specie_id,det_certainty,track_certainty,rec_certainty,contour
detection_id,,,,,,,,,,,,,,
432533639,129,136,38529590,e619a81ff91bcf8ebc69d0763ab32681#201008230800,1775,91,108,52,39,0,-1,-1,2.578500e-10,[123 108;124 108;125 108;126 108;127 108;128 ...
432533642,129,136,38529590,e619a81ff91bcf8ebc69d0763ab32681#201008230800,1778,123,88,77,51,0,-1,-1,2.616500e-07,[177 88;178 88;179 88;180 88;181 88;182 88;18...
432533647,129,136,38529590,e619a81ff91bcf8ebc69d0763ab32681#201008230800,1783,256,27,58,70,0,-1,-1,2.825500e-01,[259 27;260 27;261 27;262 27;263 27;264 27;26...
